# Rounding the better way. v.2.0

In previous posts I wrote about [rounding to nearest multiple](https://github.com/root-11/root-11.github.io/blob/master/content/round_to_nearest_n.ipynb) and [rounding of dates and times](https://github.com/root-11/root-11.github.io/blob/master/content/rounding_date_and_times.ipynb)

Here is version 2.0 of that: It rounds `floats`, `ints` and `datetimes`.

In [1]:
from datetime import date, datetime, time, timedelta, timezone
epoch = datetime(2000,1,1,0,0,0,0,timezone.utc)
epoch_no_tz = datetime(2000,1,1,0,0,0,0)

def xround(value, multiple, up=None):
    """a nicer way to round numbers.

    :param value: float, integer or datetime to be rounded.
    :param multiple: float, integer or timedelta to be used as the base of the rounding.
    :param up: None (default) or boolean rounds half, up or down.
        round(1.6, 1) rounds to 2.
        round(1.4, 1) rounds to 1.
        round(1.5, 1, up=True) rounds to 2.
        round(1.5, 1, up=False) rounds to 1.
    :return: rounded value

    Examples:

    [1] multiple = 1 is the same as rounding to whole integers.
    [2] multiple = 0.001 is the same as rounding to 3 digits precision.
    [3] mulitple = 3.1415 is rounding to nearest multiplier of 3.1415
    [4] value = datetime(2022,8,18,11,14,53,440)
    [5] multiple = timedelta(hours=0.5)
    [6] xround(value,multiple) is datetime(2022,8,18,11,0)
    """
    epoch = 0
    if isinstance(value, (datetime)) and isinstance(multiple, timedelta):
        if value.tzinfo is None:
            epoch = epoch_no_tz
        else:
            epoch = epoch

    value2 = value - epoch
    if value2 == 0:
        return value2

    low = (value2 // multiple) * multiple
    high = low + multiple
    if up is True:
        return high + epoch
    elif up is False:
        return low + epoch
    else:
        if abs((high + epoch) - value) < abs(value - (low + epoch)):
            return high + epoch
        else:
            return low + epoch

In [2]:
xround(0,1,up=True)

0

And now to the tests.

In [3]:
import math
# round up
assert xround(0,1,True) == 0
assert xround(1.6, 1, True) == 2
assert xround(1.4, 1, True) == 2
# round down
assert xround(0,1,False) == 0
assert xround(1.6, 1, False) == 1
assert xround(1.4, 1, False) == 1
# round half
assert xround(0,1) == 0
assert xround(1.6, 1) == 2
assert xround(1.4, 1) == 1

# round half
assert xround(16, 10) == 20
assert xround(14, 10) == 10

# round half
assert xround(-16, 10) == -20
assert xround(-14, 10) == -10

# round to odd multiples
assert xround(6, 3.1415, 1) == 2 * 3.1415

assert xround(1.2345, 0.001, True) == 1.2349999999999999 and math.isclose(1.2349999999999999, 1.235)
assert xround(1.2345, 0.001, False) == 1.234

assert xround(123, 100, False) == 100
assert xround(123, 100, True) == 200

assert xround(123, 5.07, False) == 24 * 5.07

dt = datetime(2022,8,18,11,14,53,440)

td = timedelta(hours=0.5)    
assert xround(dt,td, up=False) == datetime(2022,8,18,11,0)
assert xround(dt,td, up=None) == datetime(2022,8,18,11,0)
assert xround(dt,td, up=True) == datetime(2022,8,18,11,30)

td = timedelta(hours=24)
assert xround(dt,td, up=False) == datetime(2022,8,18)
assert xround(dt,td, up=None) == datetime(2022,8,18)
assert xround(dt,td, up=True) == datetime(2022,8,19)


td = timedelta(days=0.5)
assert xround(dt,td, up=False) == datetime(2022,8,18)
assert xround(dt,td, up=None) == datetime(2022,8,18,12)
assert xround(dt,td, up=True) == datetime(2022,8,18,12)

td = timedelta(days=1.5)
assert xround(dt,td, up=False) == datetime(2022,8,18)
assert xround(dt,td, up=None) == datetime(2022,8,18)
assert xround(dt,td, up=True) == datetime(2022,8,19,12)

td = timedelta(seconds=0.5)
assert xround(dt,td, up=False) == datetime(2022,8,18,11,14,53,0)
assert xround(dt,td, up=None) == datetime(2022,8,18,11,14,53,0)
assert xround(dt,td, up=True) == datetime(2022,8,18,11,14,53,500000)

td = timedelta(seconds=40000)
assert xround(dt,td, up=False) == datetime(2022,8,18,6,40)
assert xround(dt,td, up=None) == datetime(2022,8,18,6,40)
assert xround(dt,td, up=True) == datetime(2022,8,18,17,46,40)

Here's an example determining intervals:

In [4]:
td = timedelta(days=2)
for i in range(10):
    b = datetime(2022,8,18,6,40) + timedelta(days=i*12/24)
    a = xround(b, td, up=False)
    c = xround(b, td, up=True)
    print(a,"<",b, "<", c)

2022-08-17 00:00:00 < 2022-08-18 06:40:00 < 2022-08-19 00:00:00
2022-08-17 00:00:00 < 2022-08-18 18:40:00 < 2022-08-19 00:00:00
2022-08-19 00:00:00 < 2022-08-19 06:40:00 < 2022-08-21 00:00:00
2022-08-19 00:00:00 < 2022-08-19 18:40:00 < 2022-08-21 00:00:00
2022-08-19 00:00:00 < 2022-08-20 06:40:00 < 2022-08-21 00:00:00
2022-08-19 00:00:00 < 2022-08-20 18:40:00 < 2022-08-21 00:00:00
2022-08-21 00:00:00 < 2022-08-21 06:40:00 < 2022-08-23 00:00:00
2022-08-21 00:00:00 < 2022-08-21 18:40:00 < 2022-08-23 00:00:00
2022-08-21 00:00:00 < 2022-08-22 06:40:00 < 2022-08-23 00:00:00
2022-08-21 00:00:00 < 2022-08-22 18:40:00 < 2022-08-23 00:00:00
